# Django log 和 restful

## 实现系统自带的登录，登出的方法

In [ ]:
# settings.py 中 
# 没登录的跳转路径，将原来的默认路径覆盖
LOGIN_URL = '/uauth/dj_login/'

In [ ]:
# urls.py中
# 从下面这个包中导入login_required这个方法
from django.contrib.auth.decorators import login_required
urlpatterns = [
    # 使用这个方法可以让在访问这个路由的时候，检查是否有sessionid，如果没有自动跳转页面
    url(r'^index/', login_required(views.index))
]

In [ ]:
from django.contrib import auth


def djlogin(request):
    if request.method == 'GET':
        return render(request, 'day6_login.html')
    if request.method == 'POST':
        name = request.POST.get('name')
        password = request.POST.get('password')
    # 验证用户名和密码，通过的话，返回user
    user = auth.authenticate(username=name, password=password)
    if user:
        # 绑定sessionID
        auth.login(request, user)
        return HttpResponseRedirect('/stu/index/')
    else:
        return render(request, 'day6_login.html')
    
    
def djregist(request):
    if request.method == 'GET':
        return render(request, 'register.html')
    if request.method == 'POST':
        name = request.POST.get('name')
        password = request.POST.get('password')
        # 创建系统自带的user的方法
        User.objects.create_user(username=name, password=password)
        return HttpResponseRedirect('/uauth/dj_login/')
    

def djlogout(request):

    if request.method == 'GET':

        auth.logout(request)
        return HttpResponseRedirect('/uauth/dj_login/')

## 实现访问统计
- 将中间件加入到settings中
- 创建一个存数据的表

In [ ]:
# 在utils下，创建VisitTimesMiddleware.py文件
from django.utils.deprecation import MiddlewareMixin

from stu.models import Visit

class VisitTimes(MiddlewareMixin):

    def process_request(self, request):

        # 统计访问的URL以及次数
        path = request.path
        # 也可以用get_or_create方法
        try:
            visit = Visit.objects.get(v_url=path)
            if visit:
                visit.v_times += 1
            visit.save()
        except Exception as e:
            print(e)
            Visit.objects.create(v_url=path, v_times=1)

## log
1. 日志logging模块
    - logging模块可以收集记录错误，警告等调试信息，在程序中可以捕获这些信息，
    - 并且甚至可以将错误的重要信息等都可以通过邮件发送给开发者
2. logging的组成
    - Loggers Logger 为日志系统的入口。每个logger 是一个具名的容器，可以向它写入需要处理的消息。
    - Handlers Handler 决定如何处理logger 中的每条消息。它表示一个特定的日志行为。
    - Filters Filter 用于对从logger 传递给handler 的日志记录进行额外的控制。
    - Formatters 日志记录需要转换成文本。Formatter 表示文本的格式
3. 日志级别
    - DEBUG：用于调试目的的底层系统信息
    - INFO：普通的系统信息
    - WARNING：表示出现一个较小的问题。
    - ERROR：表示出现一个较大的问题。
    - CRITICAL：表示出现一个致命的问题。
    - 日志级别等级CRITICAL > ERROR > WARNING > INFO > DEBUG > NOTSET

In [ ]:
# 在settings.py中
# 创建日志的路径
LOG_PATH = os.path.join(BASE_DIR, 'log')
# 如果该路径下不存在这个文件夹，将自动创建
if not os.path.isdir(LOG_PATH):
    os.mkdir(LOG_PATH)

LOGGING = {
    'version': 1,
    # True表示禁用loggers
    'disable_existing_loggers': False,

    'formatters': {
        'default': {
            'format': '%(levelno)s %(levelname)s %(funcName)s %(module)s %(asctime)s %(message)s'
        },
        'simple': {
            'format': '%(levelname)s %(module)s %(asctime)s %(message)s'
        }
    },

    'handlers': {
        'stu_handlers': {
            'level': 'DEBUG',
            #  日志文件指定为5M，超过5M重新备份，然后写入新的日志文件
            'class': 'logging.handlers.RotatingFileHandler',
            'maxBytes': 5 * 1024 * 1024,
            # 文件地址
            'filename': '%s/log.txt' % LOG_PATH,
            'formatter': 'default'
        },
        'uauth_handlers': {
            'level': 'DEBUG',
            #  日志文件指定为5M，超过5M重新备份，然后写入新的日志文件
            'class': 'logging.handlers.RotatingFileHandler',
            'maxBytes': 5 * 1024 * 1024,
            # 文件地址
            'filename': '%s/uauth_log.txt' % LOG_PATH,
            'formatter': 'simple'
        }
    },
    'loggers': {
        'stu': {
            'handlers': ['stu_handlers'],
            'level': 'INFO'
        },
        'auth': {
            'handlers': ['uauth_handlers'],
            'level': 'INFO'
        }
    },
    'filter': {}
}


In [ ]:

from django.utils.deprecation import MiddlewareMixin

from stu.models import Visit
# 注意导入的是python自带的模块
import logging
# 指定一个设置好的loggers
logger = logging.getLogger('auth')
class VisitTimes(MiddlewareMixin):

    def process_request(self, request):

        # 统计访问的URL以及次数
        path = request.path
        try:
            visit = Visit.objects.get(v_url=path)
            if visit:
                visit.v_times += 1
            visit.save()
        except Exception as e:
            print(e)
            # 应用，打印出来的error级别的信息
            logger.error(e)
            Visit.objects.create(v_url=path, v_times=1)

In [ ]:
from django.core.paginator import Paginator
from django.http import HttpResponseRedirect
from django.shortcuts import render
from django.core.urlresolvers import reverse

# Create your views here.
from stu.models import Student, StudentInfo
from uauth.models import Users
from stu.serializers import StudentSerializer
import logging

logger = logging.getLogger('stu')


def index(request):

    if request.method == 'GET':
        stuinfos = StudentInfo.objects.all()
        # 打印出info级别的日志信息
        logger.info('url: %s method: %s 获取学生信息成功' % (request.path, request.method))
        return render(request, 'index.html', {'stuinfos': stuinfos})

## restful 
- REST是所有Web应用都应该遵守的架构设计指导原则。
- Representational State Transfer，翻译是”表现层状态转化”。
- 面向资源是REST最明显的特征，对于同一个资源的一组不同的操作。

1. api定义规范
- http://xxx.com/api/

2. 资源
- 在RESTful架构中，每个网址代表一种资源（resource），所以网址中不能有动词，只能有名词，而且所用的名词往往与数据库的表格名对应。一般来说，数据库中的表都是同种记录的"集合"（collection），所以API中的名词也应该使用复数。
3. http请求方式
    - GET（SELECT）：从服务器取出资源（一项或多项）
    - POST（CREATE）：在服务器新建一个资源
    - PUT（UPDATE）：在服务器更新资源（客户端提供改变后的完整资源）
    - PATCH（UPDATE）：在服务器更新资源（客户端提供改变的属性）
    - DELETE（DELETE）：从服务器删除资源
4. 状态码

In [ ]:
"""
200 OK - [GET]：服务器成功返回用户请求的数据，该操作是幂等的（Idempotent）。

201 CREATED - [POST/PUT/PATCH]：用户新建或修改数据成功。

202 Accepted - [*]：表示一个请求已经进入后台排队（异步任务）

204 NO CONTENT - [DELETE]：用户删除数据成功。

400 INVALID REQUEST - [POST/PUT/PATCH]：用户发出的请求有错误，服务器没有进行新建或修改数据的操作，该操作是幂等的。

401 Unauthorized - [*]：表示用户没有权限（令牌、用户名、密码错误）。

403 Forbidden - [*] 表示用户得到授权（与401错误相对），但是访问是被禁止的。

404 NOT FOUND - [*]：用户发出的请求针对的是不存在的记录，服务器没有进行操作，该操作是幂等的。

406 Not Acceptable - [GET]：用户请求的格式不可得（比如用户请求JSON格式，但是只有XML格式）。

410 Gone -[GET]：用户请求的资源被永久删除，且不会再得到的。

422 Unprocesable entity - [POST/PUT/PATCH] 当创建一个对象时，发生一个验证错误。

500 INTERNAL SERVER ERROR - [*]：服务器发生错误，用户将无法判断发出的请求是否成功。

"""

In [ ]:
5. 安装
    - pip install djangorestframework
    - pip install django-filter

In [ ]:
# settings.py中
INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'stu',
    'uauth',
    'rest_framework' # 将这个加入到app中名字不要改变
]

In [ ]:
# 在一个app中创建serializers.py文件
from rest_framework import serializers
from stu.models import Student

class StudentSerializer(serializers.ModelSerializer):

    class Meta:
        model = Student
        fields = ['id', 's_name', 's_tel']

    def to_representation(self, instance):

        data = super().to_representation(instance)
        try:
            data['s_addr'] = instance.studentinfo.i_addr
        except Exception as e:
            data['s_addr'] = ''

        return data

In [ ]:
# 在urls.py中
from django.conf.urls import url
from django.contrib.auth.decorators import login_required

from rest_framework.routers import SimpleRouter

from stu import views

router = SimpleRouter()
router.register(r'student', views.StudentEdit)

urlpatterns = [
    url(r'^index/', login_required(views.index)),
    url(r'^addstu/', views.addStu, name='addStu'),
    url(r'^addstuinfo/(?P<stu_id>\d+)/', views.addStuInfo, name='addStuInfo'),
    url(r'^astupage/', views.aStuPage),
    url(r'^showstus/', views.showStus)

]
urlpatterns += router.urls

In [ ]:
# 在views.py中
from django.core.paginator import Paginator
from django.http import HttpResponseRedirect
from django.shortcuts import render
from django.core.urlresolvers import reverse

from rest_framework import mixins, viewsets


# 继承这几个类之后，就不需要写增删改查
class StudentEdit(mixins.ListModelMixin,  # 获取所有数据
                  mixins.RetrieveModelMixin,  # 获取某一个数据
                  mixins.UpdateModelMixin,  # 更新数据，可以使用put和patch
                  mixins.DestroyModelMixin,  # 删除数据，可以是用delete
                  mixins.CreateModelMixin,  # 创建数据，可以使用post方法
                  viewsets.GenericViewSet):

    # 查询所有信息
    queryset = Student.objects.all()
    # 序列化
    serializer_class = StudentSerializer